In [1]:
import json, gzip, tqdm, math, csv, random, numpy as np, pandas as pd, os
from matplotlib import pyplot as plt
import matplotlib.lines as lines
from collections import defaultdict
from sklearn import linear_model
from nltk.stem.porter import *
from nltk.corpus import stopwords
from string import punctuation
from IPython.display import display, HTML

In [2]:
def load(base_path):
    os.makedirs(base_path, exist_ok=True)
    meta, rev = pd.read_json(os.path.join(base_path, "meta.json")), pd.read_json(os.path.join(base_path, "rev.json"))
    return meta, rev

In [3]:
meta, rev = load("./data/shoes")
meta = meta.reset_index()
meta["index"] = meta.index
rev = rev.reset_index()
rev["index"] = rev.index

In [4]:
print(len(meta), len(rev))
display(meta[:1])
display(rev[:1])

57985 165947


,index,asin,related,title,price,salesRank,imUrl,brand,categories,description,feats
0,0,8921463216,"{'also_viewed': ['8921463267', '8921463208', '...",Hello Kitty LALA Lovely Womens Summer Slippers...,NaN,{'Beauty': 377777},http://ecx.images-amazon.com/images/I/41hbUxgB...,None,"[Clothing, Shoes & Jewelry, Women, Shoes, Slip...",None,"[0.0, 3.3559999466, 0.0, 0.0, 1.5535999537, 0...."


,index,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A2IQ3AQHJNJUNZ,8921463216,GirlGamer,"[0, 0]","Everbody loves my sandals, a little narrow aro...",5,Love Love Love,1395014400,"03 17, 2014"


In [5]:
i, a = meta["index"], meta["asin"]
asin_to_idx = dict(zip(a, i))

In [6]:
rPU = defaultdict(list)
# positive pair list:
for idx, b in tqdm.tqdm(rev.iterrows()):
    asin, user = b["asin"], b["reviewerName"]
    rPU[user].append(asin)
for ratList in rPU.values():
    ratList.sort()
# 121084 - name, 146593 - user
print(len(rPU))

165947it [00:07, 21822.20it/s]

121084


In [7]:
all_items = set(meta["asin"])
positives = set()
for user, rats in tqdm.tqdm(rPU.items()):
    for i in range(len(rats)):
        for j in range(i + 1, len(rats)):
            positives.add((rats[i], rats[j]))
positives_li = list(positives)
ones = [1] * len(positives_li)

100%|██████████| 121084/121084 [00:01<00:00, 105427.82it/s]


In [8]:
from sklearn.model_selection import train_test_split

# 80 - 10 - 10
# 90 - 10
# === THESE ARE ONLY POSITIVES SAMPLES
X_tr, X_te, y_tr, y_te = train_test_split(positives_li, ones, test_size=0.1, random_state=1)
X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr, test_size=0.111, random_state=1) # 0.25 x 0.8 = 0.2
print("Num positives", len(positives), "Maximum possible (using combinatorics)", sum([len(t) * (len(t) - 1) / 2 for t in rPU.values() if len(t) > 1]))
print(f"X_tr: {len(X_tr)}, X_va: {len(X_va)}, X_te: {len(X_te)}")

Num positives 2140082 Maximum possible (using combinatorics) 3687163.0
X_tr: 1712278, X_va: 213795, X_te: 214009


In [9]:
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pytorch_lightning as pl
import sklearn
import random


In [10]:
class PosNegDataset(Dataset):
    # pos_list: positives in dataset
    # asins: all asins in dataset
    # all_pos: all positives in the entire dataset
    # features: all features as df
    # asin_to_idx: map asin to an idx in features
    def __init__(self, pos_list, all_pos, all_asins, features, asin_to_idx):
        self.pos_list = pos_list
        self.all_pos = all_pos
        self.all_asins = all_asins
        self.features = features
        self.asin_to_idx = asin_to_idx
    
    def __len__(self):
        return len(self.pos_list) * 2

    def get_feature(self, asin):
        idx = self.asin_to_idx[asin]
        return torch.tensor(self.features.loc[idx]["feats"])
    def generate_neg_pair(self):
        pair = random.sample(self.all_asins, 2)
        pair.sort()
        pair = tuple(pair)
        if pair in self.all_pos:
            pair = random.sample(self.all_asins, 2)
            pair.sort()
            pair = tuple(pair)
        return pair
    def __getitem__(self, idx):
        if idx < len(self.pos_list):
            a, b = self.pos_list[idx]
            return self.get_feature(a), self.get_feature(b), 1
        a, b = self.generate_neg_pair()
        return self.get_feature(a), self.get_feature(b), 0

In [16]:
class Mahalanobis(pl.LightningModule):
    def __init__(self, embedding_dims = 4096, K = 10, c = 2):
        super().__init__()
        self.mahal = nn.Linear(embedding_dims, K)
        self.l = nn.BCEWithLogitsLoss()
        self.c = c
    def forward(self, user_input, item_input):
        # bs x k
        a = self.mahal(user_input - item_input)
        b = torch.linalg.norm(a, dim=1)
        out = b - self.c
        return out
    def training_step(self, batch, batch_idx):
        i1, i2, l = batch
        pred = self(i1, i2)
        loss = self.l(pred, l.float())
        self.log("tr/loss_step", loss)
        return loss
    def validation_step(self, batch, batch_idx):
        i1, i2, l = batch
        pred = self(i1, i2)
        loss = self.l(pred, l.float()).item()
        accuracy = sklearn.metrics.accuracy_score(l.cpu(), pred.cpu().flatten() > 0.0)
        self.log("val/loss_step", loss)
        self.log("val/acc_step", accuracy)
        return {"val_acc": accuracy, "val_loss": loss, "pred": pred.cpu().flatten().numpy(), "label": l.cpu().flatten().numpy()}
    def training_epoch_end(self, training_step_outputs):
        avg_loss = np.mean(np.mean(training_step_outputs))
        self.log("tr/loss_epoch", avg_loss)
    def validation_epoch_end(self, validation_step_outputs):
        acc, loss, pred, label = zip(*[(t["val_acc"], t["val_loss"], t["pred"], t["label"]) for t in validation_step_outputs])
        mean_acc, mean_loss = np.mean(acc), np.mean(loss)
        pred, label = np.array(pred).flatten(), np.array(label).flatten()
        self.log("val/loss_epoch", mean_loss)
        self.log("val/acc_epoch", sklearn.metrics.accuracy_score(label, pred > 0.0))
        self.log("val/f1_epoch", sklearn.metrics.f1_score(label, pred > 0.0))
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(PosNegDataset(X_tr, positives, all_items, meta, asin_to_idx),
                          batch_size=128, num_workers=0, shuffle=True)
    def val_dataloader(self):
        return DataLoader(PosNegDataset(X_va, positives, all_items, meta, asin_to_idx),
                          batch_size=128, num_workers=0, shuffle=True)
    
        

In [12]:
tr_ds = PosNegDataset(X_tr, positives, all_items, meta, asin_to_idx)
tr_dl = DataLoader(tr_ds, batch_size = 25, num_workers = 0)
for i, j, l in tr_dl:
    print(i[0], j[0], l[0])
    break

tensor([0.3460, 0.0000, 0.5135,  ..., 0.0000, 0.0000, 0.0000]) tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.8614, 0.0000]) tensor(1)


In [17]:
from pytorch_lightning.callbacks import ModelCheckpoint
model = Mahalanobis()
checkpoint_callback = ModelCheckpoint(monitor="val/acc_step")
trainer = pl.Trainer(max_epochs=10, gpus=1, reload_dataloaders_every_epoch=True, progress_bar_refresh_rate=50, logger=True, default_root_dir="./models", callbacks=[checkpoint_callback])
trainer.fit(model)

C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:89: LightningDeprecationWarning: `reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6. Please use `reload_dataloaders_every_n_epochs` in Trainer.
  "`reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6."
GPU available: True, used: True
TPU availabl

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\data_loading.py:442: UserWarning: Your `val_dataloader` has `shuffle=True`,it is strongly recommended that you turn this off for val/test/predict dataloaders.
  f"Your `{mode.dataloader_prefix}_dataloader` has `shuffle=True`,"
C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Epoch 0:   0%|          | 100/30096 [00:14<1:13:41,  6.78it/s, loss=0.667, v_num=2]

C:\Users\James\anaconda3\envs\cse158_37\lib\site-packages\pytorch_lightning\trainer\trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
